# Computing and Visualizing Access and Equity

In this workbook we will combine the travel times we calculated in the previous notebook with the demographics visualized in the first to get an understanding of how Calgary's transit access to two opportunities (hospitals and childcare) are distributed.

Our basic workflow will be this:
1. Using the travel time matrices and destination data, calculate a metric for each DA zone
2. Compute a population weighted sum for different population groups to see how these benefits are distributed on average.

In [1]:
import pandas as pd
import geopandas as gpd

demographics = pd.read_csv("data/demographics.csv", dtype={"dauid":str})
demographics.head()

,dauid,area_sq_km,pop_total,hhld_total,vismin_total,vismin_vismin,lico_total,lico_lico,fam_total,fam_oneparent,fam_onemother
0,48060056,0.1221,465,166,480.0,265.0,465.0,15.0,140.0,15.0,15.0
1,48060057,0.0866,377,134,390.0,155.0,375.0,10.0,120.0,15.0,15.0
2,48060058,0.1557,467,183,475.0,255.0,470.0,30.0,140.0,25.0,15.0
3,48060059,0.0888,351,144,385.0,175.0,350.0,15.0,110.0,5.0,5.0
4,48060060,0.2522,612,215,635.0,345.0,615.0,20.0,195.0,30.0,15.0


next, lets load in our travel time matrix for the AM peak with access to hospitals:

In [2]:
hosp_am = pd.read_csv("data/mx_hospitals_am.csv", dtype={"from_id":str})
hosp_am.head()

,from_id,to_id,travel_time
0,48060056,0,71.0
1,48060056,1,72.0
2,48060056,2,85.0
3,48060056,3,NaN
4,48060056,4,64.0


We have a decision to make here: What exactly should we do with the `NaN` values? Let's for now fill them in with a 1-hour extra value:

In [5]:
hosp_am["travel_time"] = hosp_am["travel_time"].fillna(180)
# Now we get minimal travel times
hosp_am = hosp_am[["from_id", "travel_time"]].groupby("from_id", as_index=False).min()
hosp_am.head()

,from_id,travel_time
0,48060056,64.0
1,48060057,61.0
2,48060058,61.0
3,48060059,63.0
4,48060060,63.0
